# 1 Import Dataset
The first step is to import the dataset and have a look at which columns contain a lot of data points

Please note that the program expects the dataset to be saved as latestdata.csv in the same folder as the program.

In [ ]:
import pandas as pd

# Read in the dataset
cov = pd.read_csv("latestdata.csv")
print('Read dataset from latestdata.csv')

# Inspect data
print(cov.count())

# 2 Data Preparation

## 2.1 Drop irrelevant columns and rows

In [ ]:
# Identify the columns required for the problem
fields = ["outcome", "age", "sex", "date_confirmation", "date_onset_symptoms", "country"] # 3493 rows - mostly from the phillipines
dataset = cov[fields]
 
# Drop the rows which are missing information
dataset = dataset.dropna(subset=fields)

# Store the set for future use
dataset.to_csv("dataset.csv")
print('Stored dataset as dataset.csv')

In [ ]:
# Instead of doing the above steps you can load the processed dataset
import pandas as pd
dataset = pd.read_csv('dataset.csv')
print('Read dataset from dataset.csv')
# Drop the unnamed column
dataset = dataset.drop(columns=['Unnamed: 0'])

print(dataset.count())

## 2.2 Feature encoding

In [ ]:
# Tidy the outcome column
dataset = dataset.replace(to_replace={'outcome': {
    'died':0,
    'death':0,
    'Deceased':0,
    'dead':0,
    'stable':1,
    'treated in an intensive care unit (14.02.2020)':1,
    'Symptoms only improved with cough. Currently hospitalized for follow-up.':1, # TODO drop and compare results
    'severe':0,        
    'Hospitalized':1, # TODO drop and compare results
    'discharge':1,
    'discharged':1,
    'Discharged':1,
    'Alive':1,
    'recovered':1,
    }})

# Tidy the ages column
dataset = dataset.replace(to_replace={'age': {
    '0-9':5,
    '10-19':15,
    '20-29':25,
    '30-39':35,
    '40-49':45,
    '50-59':55,
    '60-69':65,
    '70-79':75,
    '80-89':85,
    '90-99':95,
    }}, regex=True)

# Apply feature scaling to the ages
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dataset[['age']] = scaler.fit_transform(dataset[['age']])

# Replace the two dates columns with days_waiting
gaps = []
from datetime import date, datetime, timedelta
for i in range(len(dataset['date_confirmation'])):
    dataset['date_confirmation'][i] = datetime.strptime(dataset['date_confirmation'][i], r'%d.%m.%Y')
    dataset['date_onset_symptoms'][i] = datetime.strptime(dataset['date_onset_symptoms'][i], r'%d.%m.%Y')
    gaps.append(dataset['date_confirmation'][i] - dataset['date_onset_symptoms'][i])

dataset['days_waiting'] = gaps
dataset = dataset.drop(columns=['date_confirmation', 'date_onset_symptoms'])
dataset['days_waiting'] = dataset['days_waiting'].dt.days
dataset = dataset[dataset['days_waiting'] >= 0]

# Encode the sex data as integers
dataset = dataset.replace(to_replace={'sex': {
    'male':0,
    'female':1
}})

# Encode the country data using one hot encoding
countries_df = pd.get_dummies(dataset['country'])
dataset = pd.concat([dataset, countries_df], axis=1)
dataset = dataset.drop(columns=['country'])

# Save the cleaned up dataset for future use
dataset.to_csv('dataset_clean.csv')
print('Saved dataset as dataset_clean.csv')

# 3 Visualisation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,3, figsize=(20,5))

data = dataset['age']
sns.histplot(data=data, ax=ax[0])

data = dataset.loc[dataset['outcome'] == 0, 'age']
sns.histplot(data=data, ax=ax[1]).set_title('Died')
data = dataset.loc[dataset['outcome'] == 1, 'age']
sns.histplot(data=data, ax=ax[2]).set_title('Recovered')

print('Produced graphs to show the distribution of the scaled ages for the dataset, those that died and those that recovered')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,5))

data = dataset['sex']
sns.histplot(data=data, ax=ax[0])

data = dataset.loc[dataset['outcome'] == 0, 'sex']
sns.histplot(data=data, ax=ax[1]).set_title('Died')
data = dataset.loc[dataset['outcome'] == 1, 'sex']
sns.histplot(data=data, ax=ax[2]).set_title('Recovered')

print('Produced graphs to show the distribution of the genders for the dataset, those that died and those that recovered')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,5))

data = dataset['days_waiting']
sns.histplot(data=data, ax=ax[0])

data = dataset.loc[dataset['outcome'] == 0, 'days_waiting']
sns.histplot(data=data, ax=ax[1]).set_title('Died')
data = dataset.loc[dataset['outcome'] == 1, 'days_waiting']
sns.histplot(data=data, ax=ax[2]).set_title('Recovered')

print('produced graphs to show the distribution of the number of days between the onset of symptoms and the date of confirmation for the dataset, those that died and those that recovered')

# 4 Model Training

## 4.1 Split the dataset into features/labels and test/train

In [ ]:
# Split the data into features and labels
features = dataset.drop(['outcome'], axis=1)
labels = dataset['outcome']

# print('features\n', features.value_counts())
# print('labels\n', labels.value_counts())

# Split the data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=0) # This also shuffles the data

## 4.2 Fit the models
In this section we fit 6 different models for comparison. From this 6 I will choose the best performing 3.

In [ ]:
# Import and fit a Support Vector Machine
from sklearn import svm
svm_classifier = svm.SVC(gamma='auto', class_weight={0: 2832/624, 1:1})
svm_classifier.fit(X_train, y_train)

# Immport and fit a logistic regression model
from sklearn import linear_model
# LR_classifier = linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000) # MULTINOMIAL There are five solvers that can be used to obtain the weights 
LR_classifier = linear_model.LogisticRegression(solver='lbfgs', max_iter=1000, class_weight={0: 2832/624, 1:1})
LR_classifier.fit(X_train, y_train)

# Import and fit a decision tree
from sklearn import tree
tree_classifier = tree.DecisionTreeClassifier(class_weight={0: 2832/624, 1:1})
tree_classifier.fit(X_train, y_train)

# Import a random forest
from sklearn.ensemble import RandomForestClassifier
forest_classifier = RandomForestClassifier(class_weight={0: 2832/624, 1:1}, n_estimators=100)
forest_classifier.fit(X_train, y_train)

# Import and fit a naive bayes model
from sklearn.naive_bayes import GaussianNB
# nb_classifier = GaussianNB(priors=[624/3456, 2832/3456])
nb_classifier = GaussianNB(priors=[2832/3456, 624/3456])
nb_classifier.fit(X_train, y_train)

# # import and fit an XGboost model
import xgboost.sklearn as xgb
xgb_classifier = xgb.XGBClassifier(class_weight={0: 2832/624, 1:1}, use_label_encoder=False, verbosity=0)
xgb_classifier.fit(X_train, y_train)

print('Trained a selection of models on the training data')

# 5 Model Evaluation

In this section we look at how the models performed on the test set. 
This allows us to select the top 3 models before attempting to improve their performance through hyperparameter tuning.

The evaluation metrics of interest at this step are the accuracy, balanced_accuracy and f1_score

## 5.1 Use the models to make some predictions

In [ ]:
# Make predictions on the test dataset with each model
svm_predictions = svm_classifier.predict(X_test)
LR_predictions = LR_classifier.predict(X_test)
tree_predictions = tree_classifier.predict(X_test)
forest_predictions = forest_classifier.predict(X_test)
nb_predictions = nb_classifier.predict(X_test)
xgb_predictions = xgb_classifier.predict(X_test)


## 5.2 Measure the accuracy of the predictions

The accuracy is immediately easy to interpret and is therefore worth examining before looking at other metrics

In [ ]:
# Measure the accuracy of each set of predictions
from sklearn import metrics
svm_accuracy = metrics.accuracy_score(y_test, svm_predictions)
LR_accuracy = metrics.accuracy_score(y_test, LR_predictions)
tree_accuracy = metrics.accuracy_score(y_test, tree_predictions)
forest_accuracy = metrics.accuracy_score(y_test, forest_predictions)
nb_accuracy = metrics.accuracy_score(y_test, nb_predictions)
xgb_accuracy = metrics.accuracy_score(y_test, xgb_predictions)

print(f'SVM score: {svm_accuracy}\nLR score: {LR_accuracy}\nTree score: {tree_accuracy}\nForest score: {forest_accuracy}\nNaive Bayes score: {nb_accuracy}\nXGB score: {xgb_accuracy}')


## 5.3 View more detailed classification reports

In [ ]:
# Produce classification reports

# precision = % of Positives that are correct and recall = % of negatives that are found)
# The recall for a class is the sensitivity to that class (if it exists is it identified)
# The precision for a class is the 

def evaluate(model_name, predictions):
    print('\033[95m' + model_name.upper() + '\033[0m')
    scores = pd.DataFrame()
    accuracy = metrics.accuracy_score(y_test, predictions)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, predictions)
    f1score = metrics.f1_score(y_test, predictions)
    scores[model_name] = [accuracy, balanced_accuracy, f1score]
    scores.index = ['Accuracy', 'Balanced Accuracy', 'f1 Score']
    print(scores)
    print(f'\nConfusion matrix for {model_name}:')
    print(metrics.confusion_matrix(y_test, predictions))
    print(f'\nFull classification report for {model_name}:')
    print(metrics.classification_report(y_test, predictions, target_names=['died','recovered']))
    print('\n')


evaluate('SVM', svm_predictions)
evaluate('Logistic Regression', LR_predictions)
evaluate('Decision tree', tree_predictions)
evaluate('Random forest', forest_predictions)
evaluate('Naive Bayes', nb_predictions)
evaluate('XGBoost', xgb_predictions)

At this point we can see that the f1 score.....?

# 6 Model Improvement

This section involves trying a cross validation method and hyperparameter tuning to try and improve the models
The tuning actually occurs during the .fit() call on the GridSearchCV object.
Calling this on the training data means cross validation folds are automatically generated from the training data and the test data remains unseen (to be used for evaluation later).
Because of this, we can see that the accuracy of the models is lower than the previous accuracy but we hope that the model quality has improved regardless. This will be tested at the end.

For each model, we want to find the best set of parameters to improve the ____.

We want to choose from:
- 'accuracy'
- 'balanced_accuracy'
- 'f1'
- 'f1_weighted'
- 'precision'
- 'recall'
- 'roc_auc'

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

## 6.1 Tune the random forest hyperparameters

In [ ]:
model = RandomForestClassifier()
class_weight = [{0: 2832/624, 1:1}]

n_estimators = [10, 50, 100, 200]
criterions = ['gini', 'entropy']
max_depths = [None, 10, 100]
max_featuress = ['auto', 'sqrt', 'log2']

rf_param_grid = dict(class_weight=class_weight, n_estimators=n_estimators, criterion=criterions, max_depth=max_depths, max_features=max_featuress)

# svm_grid_search = GridSearchCV(estimator=model, param_grid=svm_param_grid, n_jobs=-1, cv=5, scoring='accuracy', error_score=0, verbose=1)
rf_grid_search = RandomizedSearchCV(estimator=model, param_distributions=rf_param_grid, n_jobs=-1, cv=5, scoring='balanced_accuracy', error_score=0, verbose=1)
rf_grid_result = rf_grid_search.fit(X_train, y_train)
print(f"Best: {rf_grid_result.best_score_} using {rf_grid_result.best_params_}")

## 6.1 Tune the SVM hyperparameters
We will tune the kernel (and where relevant the gamma value) as well as the C values

In [ ]:
# Tune SVM hyperparameters
model = svm.SVC()
# print(model.get_params())
class_weight = [{0: 2832/624, 1:1}]
kernels = ['rbf', 'linear']
gammas = ['auto', 'scale']
c_values = [100, 10, 1, 0.1, 0.01]

svm_param_grid = dict(class_weight=class_weight, kernel=kernels, C=c_values, gamma=gammas)
# svm_grid_search = GridSearchCV(estimator=model, param_grid=svm_param_grid, n_jobs=-1, cv=5, scoring='accuracy', error_score=0, verbose=1)
svm_grid_search = RandomizedSearchCV(estimator=model, param_distributions=svm_param_grid, n_jobs=-1, cv=5, scoring='balanced_accuracy', error_score=0, verbose=1)
svm_grid_result = svm_grid_search.fit(X_train, y_train)
print(f"Best: {svm_grid_result.best_score_} using {svm_grid_result.best_params_}")

## 6.2 Tune the hyperparameters for Logistic Regression
We will tune the solvers, C value and max_iterations parameters.

In [ ]:
# Tune logistic regression hyperparameters
model = linear_model.LogisticRegression()
# print(model.get_params())
class_weight = [2832/624, 1]
solvers = ['lbfgs', 'liblinear']
c_values = [100, 10, 1, 0.1, 0.01]
max_iter = [500, 1000]
lr_param_grid = dict(class_weight=class_weight, solver=solvers, C=c_values, max_iter=max_iter)
# lr_grid_search = GridSearchCV(estimator=model, param_grid=lr_param_grid, n_jobs=-1, cv=5, scoring='accuracy', error_score=0, verbose=1)
lr_grid_search = RandomizedSearchCV(estimator=model, param_distributions=lr_param_grid, n_jobs=-1, cv=5, scoring='accuracy', error_score=0, verbose=1)
lr_grid_result = lr_grid_search.fit(X_train, y_train)
print(f"Best: {lr_grid_result.best_score_} using {lr_grid_result.best_params_}")

## 6.3 Tune the XGBoost parameters
We tune the XGBoost hyperparameters in sections to cut running time

In [ ]:
# Tune SVM hyperparameters
model = xgb.XGBClassifier()
# print(model.get_params())

class_weight = [{0: 2832/624, 1:1}]
verbosity=[0]

max_depths = list(range(3,10)) # best was 3; default=6
min_child_weights = list(range(0,4)) # best was 3; default=1

gammas = [i/10.0 for i in range(0,5)] # default 0, best=0.2

subsamples = [i/10.0 for i in range(6,14)] # default=1, best 0.6
colsample_bytrees = [i/10.0 for i in range(6,14)] # default=1, best 0.9

reg_alphas = [0, 1e-5, 1e-2, 0.1, 1, 100] # default=0, best=100

xgb_param_grid = dict(class_weight=class_weight, max_depth=max_depths, min_child_weight=min_child_weights, gamma=gammas, subsample=subsamples, colsample_bytree=colsample_bytrees, reg_alpha=reg_alphas, verbosity=verbosity)

# xgb_grid_search = GridSearchCV(estimator=model, param_grid=xgb_param_grid, n_jobs=-1, cv=5, scoring='accuracy', error_score=0, verbose=1)
xgb_grid_search = RandomizedSearchCV(estimator=model, param_distributions=xgb_param_grid, n_jobs=-1, cv=5, scoring='balanced_accuracy', error_score=0, verbose=1)
xgb_grid_result = xgb_grid_search.fit(X_train, y_train)
print(f"Best: {xgb_grid_result.best_score_} using {xgb_grid_result.best_params_}")

# Final models

Using the best parameters determined by the hyper parameter tuning we can create a final set of classifiers

In [ ]:
svm_classifier = svm.SVC(
    class_weight={0: 2832/624, 1:1},
    kernel='linear',
    gamma='scale',
    C=0.1
)
# 0.7567 on the training data

rf_classifier = RandomForestClassifier(
    class_weight={0: 2832/624, 1:1},
    n_estimators=200,
    max_features='sqrt',
    max_depth=10,
    criterion='gini'
)
# 0.7424 on the training data

LR_classifier = linear_model.LogisticRegression(
    class_weight={0: 2832/624, 1:1},
    solver='lbfgs', 
    max_iter=500,
    C=1
)
# 0.8315 on the training data

# xgb.config_context(verbosity=0)
xgb_classifier = xgb.XGBClassifier(
    class_weight={0: 2832/624, 1:1},
    use_label_encoder=False,
    subsample=0.8,
    reg_alpha=1e-05,
    min_child_weight=0,
    max_depth=8,
    gamma=0.4,
    colsample_bytree=0.6,
    verbosity=0
)

svm_classifier.fit(X_train, y_train)
rf_classifier.fit(X_train, y_train)
LR_classifier.fit(X_train, y_train)
xgb_classifier.fit(X_train, y_train)
print('Models trained')

In [ ]:
svm_predictions = svm_classifier.predict(X_test)
forest_predictions = rf_classifier.predict(X_test)
LR_predictions = LR_classifier.predict(X_test)
xgb_predictions = xgb_classifier.predict(X_test)

evaluate('SVM', svm_predictions)
evaluate('Logistic Regression', LR_predictions)
evaluate('Random forest', forest_predictions)
evaluate('XGBoost', xgb_predictions)

# Workflow steps
- [x] Problem framing (define a problem, find a dataset and define a measure of success)
- [x] Data preparation (identify the features and split into training and testing)
- [x] Model selection (try several models and see what you get)
- [x] Model training
- [x] Model testing (evaluate the models against the measure of success)
- [x] Hyperparameter tuning (attempt to further improve the model by tuning the hyperparameters)